In [1]:
import os
import torch
from torch_geometric.data import HeteroData


filename = 'HeteroData_Learnings_v1.pt'
if os.path.exists('./'+filename):
    data = HeteroData.from_dict(torch.load('./'+filename))
    print('loading saved heterodata object')

/home/amos/mambaforge/envs/pyg_torch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading saved heterodata object


In [3]:
# get size of the data on disk in gb
import os
size = os.path.getsize('./'+filename)
print('size of file on disk: ', size/1e9, 'gb')

size of file on disk:  1.790098703 gb


In [ ]:
# sampler for Heterogeneous Graph Transformer

In [ ]:
# for each node type, add a new edge type only consisting of self loops
# this is done to allow HGT to attend to the previous node representations
# for node_type in data.node_types:
#     data[node_type, 'self_loop', node_type] = torch.cat((torch.arange(data[node_type].num_nodes),torch.arange(data[node_type].num_nodes)), dim=0)

In [1]:
# split

In [18]:
data

HeteroData(
  courses_and_programs={
    TITLE=[55796],
    DESCRIPTION=[55796],
    x=[55796, 814],
  },
  qualifications={
    TITLE=[1242],
    DESCRIPTION=[1242],
    x=[1242, 785],
  },
  skills={
    TITLE=[138698],
    x=[138698, 772],
  },
  people={ x=[293444, 24] },
  jobs={
    TITLE=[55638],
    x=[55638, 773],
  },
  organizations={
    num_nodes=13613,
    x=[13613, 2],
  },
  (skills, qualification_skill, qualifications)={ edge_index=[2, 1596] },
  (skills, course_and_program_skill, courses_and_programs)={ edge_index=[2, 258099] },
  (courses_and_programs, course_qualification, qualifications)={ edge_index=[2, 2099] },
  (courses_and_programs, course_and_programs_student, people)={
    edge_index=[2, 553454],
    edge_attr=[553454, 3],
  },
  (jobs, job_student, people)={ edge_index=[2, 293444] },
  (people, supervisor_supervisee, people)={ edge_index=[2, 217922] },
  (people, organization_student, organizations)={ edge_index=[2, 292060] },
  (jobs, job_job, jobs)={
    

In [8]:
from torch_geometric import seed_everything
import torch_geometric.transforms as T


edge_types = []
rev_edge_types = []
for edge_type in data.edge_types:
    if edge_type[1].startswith('rev_'):
        rev_edge_types.append(edge_type)
    else:
        edge_types.append(edge_type)

transform = T.RandomLinkSplit(
    is_undirected=True,
    edge_types=edge_types,
    rev_edge_types=rev_edge_types,
    num_val=0.01,
    num_test=0.01,
    add_negative_train_samples=False, # only adds neg samples for val and test, neg train are added by LinkNeighborLoader. This means for each train batch, negs. are different, for val and train they stay the same
    neg_sampling_ratio=1.0,
    disjoint_train_ratio=0, #  training edges are shared for message passing and supervision
    )

seed_everything(14)
train_data, val_data, test_data = transform(data)

def add_self_loops(data):
    for node_type in data.node_types:
        data[node_type, 'self_loop', node_type].edge_index = torch.cat((torch.arange(data[node_type].num_nodes).unsqueeze(0),torch.arange(data[node_type].num_nodes).unsqueeze(0)), dim=0)
    return data 

train_data = add_self_loops(train_data)
val_data = add_self_loops(val_data)
test_data = add_self_loops(test_data)

train_data
    

HeteroData(
  courses_and_programs={
    TITLE=[55796],
    DESCRIPTION=[55796],
    x=[55796, 814],
  },
  qualifications={
    TITLE=[1242],
    DESCRIPTION=[1242],
    x=[1242, 785],
  },
  skills={
    TITLE=[138698],
    x=[138698, 772],
  },
  people={ x=[293444, 24] },
  jobs={
    TITLE=[55638],
    x=[55638, 773],
  },
  organizations={
    num_nodes=13613,
    x=[13613, 2],
  },
  (skills, qualification_skill, qualifications)={
    edge_index=[2, 1566],
    edge_label=[1566],
    edge_label_index=[2, 1566],
  },
  (skills, course_and_program_skill, courses_and_programs)={
    edge_index=[2, 252939],
    edge_label=[252939],
    edge_label_index=[2, 252939],
  },
  (courses_and_programs, course_qualification, qualifications)={
    edge_index=[2, 2059],
    edge_label=[2059],
    edge_label_index=[2, 2059],
  },
  (courses_and_programs, course_and_programs_student, people)={
    edge_index=[2, 542386],
    edge_attr=[542386, 3],
    edge_label=[542386],
    edge_label_index=[2,

In [ ]:
train_data

In [20]:
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.sampler import NegativeSampling

num_neighbors = {}
for edge_type in data.edge_types:
    num_neighbors[edge_type] = [0,0]
    num_neighbors['skills', 'self_loops', 'skills'] = [1,0]

negative_sampling = NegativeSampling(
        mode='binary',
        amount=10  # ratio, like Graphsage
        #weight=  # "Probabilities" of nodes to be sampled: Node degree follows power law distribution
        )
num_workers = 0
    
loader = LinkNeighborLoader(
        train_data,
        num_neighbors=num_neighbors,
        edge_label_index=(('skills', 'qualification_skill', 'qualifications'), train_data['skills', 'qualification_skill', 'qualifications'].edge_label_index), # if (edge, None), None means all edges are considered
     
        neg_sampling=negative_sampling, # adds negative samples
        batch_size=64,
        shuffle=True,
        #drop_last=True,
        num_workers=num_workers,
        directed=True,  # contains only edges which are followed, False: contains full node induced subgraph
        #disjoint=True # sampled seed node creates its own, disjoint from the rest, subgraph, will add "batch vector" to loader output
        pin_memory=True, # faster data transfer to gpu
        #num_workers=2,
        #prefetch_factor=2
)
    

AttributeError: 'EdgeStorage' object has no attribute 'edge_index'

In [15]:
train_Data

NameError: name 'train_Data' is not defined

In [2]:
for x,y in zip(data.edge_types, datat.edge_types):
    print(x)
    print(data[x].edge_index.shape, datat[y].edge_index.shape)

NameError: name 'data' is not defined